In [1]:
!pip install nibabel
!pip install nilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 80.5 MB/s eta 0:00:00
  Attempting uninstall: nibabel
    Found existing installation: nibabel 3.0.2
    Uninstalling nibabel-3.0.2:
      Successfully uninstalled nibabel-3.0.2


In [57]:
import nibabel as nib
import numpy as np
from nilearn import datasets, plotting
from nilearn.image import new_img_like
from nilearn.datasets import load_mni152_template
from nilearn.image import resample_to_img, load_img
from nilearn.maskers import NiftiLabelsMasker
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
from nibabel.viewers import OrthoSlicer3D as ov
from skimage.transform import resize

In [3]:
img = nib.load('parc116.nii.gz')
template = nib.load('template.nii.gz')
image = img.get_fdata()
image.shape

(138, 176, 138)

In [4]:
data = pd.read_csv('/content/sri24_parc116.txt')
label_list=data['name']
labels = {i:label_list[i] for i in range(len(label_list))}
data.head()

,idx,name,r,g,b,scale
0,1,Precentral_L,255,128,255,255
1,2,Precentral_R,177,18,255,255
2,3,Frontal_Sup_L,45,255,47,255
3,4,Frontal_Sup_R,106,209,255,255
4,5,Frontal_Sup_Orb_L,122,255,158,255


In [5]:
labels

{0: 'Precentral_L',
 1: 'Precentral_R',
 2: 'Frontal_Sup_L',
 3: 'Frontal_Sup_R',
 4: 'Frontal_Sup_Orb_L',
 5: 'Frontal_Sup_Orb_R',
 6: 'Frontal_Mid_L',
 7: 'Frontal_Mid_R',
 8: 'Frontal_Mid_Orb_L',
 9: 'Frontal_Mid_Orb_R',
 10: 'Frontal_Inf_Oper_L',
 11: 'Frontal_Inf_Oper_R',
 12: 'Frontal_Inf_Tri_L',
 13: 'Frontal_Inf_Tri_R',
 14: 'Frontal_Inf_Orb_L',
 15: 'Frontal_Inf_Orb_R',
 16: 'Rolandic_Oper_L',
 17: 'Rolandic_Oper_R',
 18: 'Supp_Motor_Area_L',
 19: 'Supp_Motor_Area_R',
 20: 'Olfactory_L',
 21: 'Olfactory_R',
 22: 'Frontal_Sup_Medial_L',
 23: 'Frontal_Sup_Medial_R',
 24: 'Frontal_Med_Orb_L',
 25: 'Frontal_Med_Orb_R',
 26: 'Rectus_L',
 27: 'Rectus_R',
 28: 'Insula_L',
 29: 'Insula_R',
 30: 'Cingulum_Ant_L',
 31: 'Cingulum_Ant_R',
 32: 'Cingulum_Mid_L',
 33: 'Cingulum_Mid_R',
 34: 'Cingulum_Post_L',
 35: 'Cingulum_Post_R',
 36: 'Hippocampus_L',
 37: 'Hippocampus_R',
 38: 'ParaHippocampal_L',
 39: 'ParaHippocampal_R',
 40: 'Amygdala_L',
 41: 'Amygdala_R',
 42: 'Calcarine_L',
 43: '

In [39]:
masker = NiftiLabelsMasker(
  img,resampling_target=None, standardize=False, detrend=False
)
masker.fit()
report = masker.generate_report()
report

/usr/local/lib/python3.10/dist-packages/nilearn/maskers/nifti_labels_masker.py:310: UserWarning: No image provided to fit in NiftiLabelsMasker. Plotting ROIs of label image on the MNI152Template for reporting.
  warnings.warn(msg)


label value,size (in mm^3),relative size (in %)
1,23010,2.16
2,22598,2.12
3,29291,2.75
4,24806,2.33
5,4741,0.44
6,4185,0.39
7,27875,2.62
8,32024,3.01
9,6510,0.61
10,6576,0.62


In [80]:
hiv = np.load('heatmaphiv.npy')
heatmap = resize(np.array(hiv), (138,176,138))

In [81]:
temp = template.get_fdata()
flat = temp.flatten()
for i in range(len(flat)):
  if flat[i] > 0:
    flat[i] = 255
temp = np.reshape(flat, (138,176,138))

In [72]:
masked = temp*heatmap
masked_img = nib.Nifti1Image(masked, template.affine)

In [74]:
masker = NiftiLabelsMasker(
  img,resampling_target=None, standardize=False, detrend=False
)
masker.fit(masked_img)
report = masker.generate_report()
report

label value,size (in mm^3),relative size (in %)
1,23010,2.16
2,22598,2.12
3,29291,2.75
4,24806,2.33
5,4741,0.44
6,4185,0.39
7,27875,2.62
8,32024,3.01
9,6510,0.61
10,6576,0.62


In [76]:
signals = masker.transform(masked_img)[0]

In [79]:
mapping = {label_list[i]:signals[i] for i in range(len(label_list))}
{k: v for k, v in sorted(mapping.items(), key=lambda item: item[1])}

{'Vermis_3': 0.29279940517302766,
 'Cerebelum_9_L': 2.1363459799681803,
 'Vermis_1': 2.5286380520885428,
 'Cingulum_Post_L': 2.655328188509539,
 'Cingulum_Post_R': 3.09002738987416,
 'Paracentral_Lobule_R': 3.2803275146034485,
 'Supp_Motor_Area_R': 3.3180842979354384,
 'Cerebelum_9_R': 3.473787646023927,
 'Cerebelum_3_R': 4.30136722798723,
 'CblmHemiWht_R': 5.0557637541651586,
 'Cerebelum_3_L': 5.334256200691937,
 'Paracentral_Lobule_L': 6.214977814582522,
 'Vermis_2': 6.713719914637324,
 'Thalamus_R': 6.820870146254685,
 'Cingulum_Ant_R': 8.006868579966651,
 'Cerebelum_4_5_L': 8.835739900190944,
 'Supp_Motor_Area_L': 9.194951363229778,
 'Frontal_Sup_Medial_R': 9.540559569156446,
 'Cingulum_Mid_R': 9.861440415048481,
 'Postcentral_L': 11.047381013373407,
 'Cerebelum_6_L': 11.863228535154386,
 'Precentral_L': 13.00400593105174,
 'Caudate_R': 14.203783628329974,
 'Cerebelum_10_L': 14.529280225417041,
 'CblmHemiWht_L': 14.530212634544728,
 'Cerebelum_4_5_R': 14.883563008439753,
 'SupraMar

In [82]:
mci = np.load('heatmapmci.npy')
heatmap = resize(np.array(mci), (138,176,138))

In [83]:
masked = temp*heatmap
masked_img = nib.Nifti1Image(masked, template.affine)
masker = NiftiLabelsMasker(
  img,resampling_target=None, standardize=False, detrend=False
)
masker.fit(masked_img)
report = masker.generate_report()
report

label value,size (in mm^3),relative size (in %)
1,23010,2.16
2,22598,2.12
3,29291,2.75
4,24806,2.33
5,4741,0.44
6,4185,0.39
7,27875,2.62
8,32024,3.01
9,6510,0.61
10,6576,0.62


In [84]:
signals = masker.transform(masked_img)[0]
mapping = {label_list[i]:signals[i] for i in range(len(label_list))}
{k: v for k, v in sorted(mapping.items(), key=lambda item: item[1])}

{'Vermis_3': 0.3917896126914758,
 'Cerebelum_9_L': 0.9635667333453973,
 'Cingulum_Post_L': 2.153198872573396,
 'Vermis_1': 2.447178209510656,
 'Cingulum_Post_R': 2.8559804013987966,
 'Cerebelum_9_R': 3.0509845061818264,
 'Paracentral_Lobule_R': 3.198486248055399,
 'CblmHemiWht_R': 3.330286282878104,
 'Cerebelum_3_R': 3.649748681373578,
 'Supp_Motor_Area_R': 4.13426144756966,
 'Cerebelum_3_L': 5.7658794258526935,
 'Paracentral_Lobule_L': 6.1061802879028795,
 'Vermis_2': 7.089169943806251,
 'Thalamus_R': 7.493672256396913,
 'Postcentral_L': 8.161259259646815,
 'Cerebelum_6_L': 8.393671150456587,
 'Precentral_L': 9.160956664733936,
 'Cingulum_Mid_R': 9.226546814572453,
 'Cerebelum_4_5_L': 9.240069493090207,
 'Frontal_Sup_Medial_R': 9.597089299244558,
 'Cerebelum_8_L': 10.07705869089194,
 'Cingulum_Ant_R': 10.14861477361022,
 'Cerebelum_10_L': 10.581125094948462,
 'Cerebelum_Crus2_R': 12.274381583457497,
 'Cerebelum_8_R': 12.340155314985509,
 'Supp_Motor_Area_L': 12.562000936596306,
 'Cblm

In [85]:
hand = np.load('heatmaphand.npy')
heatmap = resize(np.array(hand), (138,176,138))

In [86]:
masked = temp*heatmap
masked_img = nib.Nifti1Image(masked, template.affine)
masker = NiftiLabelsMasker(
  img,resampling_target=None, standardize=False, detrend=False
)
masker.fit(masked_img)
report = masker.generate_report()
report

label value,size (in mm^3),relative size (in %)
1,23010,2.16
2,22598,2.12
3,29291,2.75
4,24806,2.33
5,4741,0.44
6,4185,0.39
7,27875,2.62
8,32024,3.01
9,6510,0.61
10,6576,0.62


In [87]:
signals = masker.transform(masked_img)[0]
mapping = {label_list[i]:signals[i] for i in range(len(label_list))}
{k: v for k, v in sorted(mapping.items(), key=lambda item: item[1])}

{'Cingulum_Post_L': 1.6542866162020222,
 'Vermis_1': 2.3021835243559505,
 'Cingulum_Post_R': 2.4538935550533982,
 'Vermis_3': 2.520827804764128,
 'Cerebelum_9_L': 3.239741867167135,
 'Cerebelum_9_R': 3.3435783969825748,
 'Cerebelum_3_L': 3.7464093996016605,
 'Supp_Motor_Area_R': 3.7993050160352704,
 'Cerebelum_3_R': 3.915250192467003,
 'Paracentral_Lobule_R': 4.179441144783516,
 'Paracentral_Lobule_L': 4.265298708847312,
 'CblmHemiWht_R': 5.5137540482280905,
 'Cerebelum_4_5_L': 5.821638886151429,
 'Cingulum_Mid_R': 6.516847924293448,
 'Thalamus_R': 6.696691391285147,
 'Supp_Motor_Area_L': 8.158140101578502,
 'Frontal_Sup_Medial_R': 9.275539012072121,
 'Vermis_2': 9.551137029786089,
 'Cerebelum_6_L': 10.763009632399584,
 'Precentral_L': 10.93383279914176,
 'Cingulum_Ant_R': 10.961568466783785,
 'CblmHemiWht_L': 12.78283960522442,
 'Cerebelum_10_L': 12.856660615972059,
 'Cerebelum_4_5_R': 12.972196388534911,
 'SupraMarginal_R': 13.476482073924949,
 'Postcentral_L': 13.66284455747444,
 'C